I need a python class which has following features:
1. it is a class for time series forecasting, 
2. it should be able to customize to any of existing or new model
3. it should be customized to any dataset
4. it should have a function to prepare the dataset, splitting the time series dataset into train and 20% for testing
5. A function for model initialization and instance creation
6. a function for training, 
7. a function for testing on 20% of data
8. test function should also have plotting of the time series dataset, and calculating the MSE, MAPE, WAPE and may be more metrics used in the time series forecasting. 
9. finally a inference function which can take as input how many steps we want to forecast in the future, 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

class TimeSeriesForecaster:
    def __init__(self, model=None):
        self.model = model
    
    def prepare_dataset(self, data, target_col, test_size=0.2):
        """
        Prepare dataset for time series forecasting by splitting into training and testing set.

        Args:
        data: pandas dataframe
        target_col: str, name of the column to be forecasted
        test_size: float, size of test set (default is 0.2)

        Returns:
        train_X: numpy array, training data (excluding target variable)
        train_y: numpy array, target variable for training data
        test_X: numpy array, test data (excluding target variable)
        test_y: numpy array, target variable for test data
        """
        self.target_col = target_col
        self.train_X, self.test_X, self.train_y, self.test_y = train_test_split(
            data.drop(columns=[self.target_col]), data[self.target_col], test_size=test_size, shuffle=False)
        
        return self.train_X, self.train_y, self.test_X, self.test_y
    
    def initialize_model(self, model_class, hyperparameters):
        """
        Initialize model for time series forecasting.

        Args:
        model_class: class object, model class to be used
        hyperparameters: dict, hyperparameters for the model

        Returns:
        None
        """
        self.model = model_class(**hyperparameters)
    
    def train(self, train_X, train_y):
        """
        Train model on the training data.

        Args:
        train_X: numpy array, training data (excluding target variable)
        train_y: numpy array, target variable for training data

        Returns:
        None
        """
        self.model.fit(train_X, train_y)
    
    def test(self, test_X, test_y):
        """
        Test model on the test data.

        Args:
        test_X: numpy array, test data (excluding target variable)
        test_y: numpy array, target variable for test data

        Returns:
        mse: float, mean squared error between predicted and actual values
        mape: float, mean absolute percentage error between predicted and actual values
        wape: float, weighted absolute percentage error between predicted and actual values
        """
        y_pred = self.model.predict(test_X)
        mse = mean_squared_error(test_y, y_pred)
        mape = np.mean(np.abs((test_y - y_pred) / test_y)) * 100
        wape = np.sum(np.abs(test_y - y_pred)) / np.sum(np.abs(test_y))
        
        # plot the time series data
        ax = test_y.plot(label='Actual')
        pd.Series(y_pred, index=test_y.index).plot(ax=ax, label='Predicted')
        ax.set_xlabel('Time')
        ax.set_ylabel('Value')
        ax.legend()
        
        return mse, mape, wape
    
    def infer(self, input_X, steps):
        """
        Predict future values based on trained model.

        Args:
        input_X: numpy array, input data (excluding target variable)
        steps: int, number of steps to forecast into the future

        Returns:
        numpy array, predicted values
        """
        for i in range(steps):
            y_pred = self.model.predict(input_X[-1].reshape(1, -1))
            input_X = np.vstack((input_X, np.append(input_X[-1][1:], y_pred)))
        
        return y_pred


yes, this seems like a good template, but I need a model as well, for example write a model in PyTorch which can do time series forecasting, the model can have following features: 
1. it will be using  a multivariate time series dataset, i.e. input  
2. the model should have a initialize function which can take some input parameters like, number of differencing, i.e. differencing used in ARIMA model
3. other configurable parameter should be number of hidden layers one would like to use in the model, buy default it will be 0 hidden layers. 
4. the model function should have a neural network, this network will take the input data frame, a multivariate time series and use fully connected neural network to build auto-regression like function. the number of past values which will used for the auto regression should be used as an input for the init function. 
5. a fit function which will do the minimization 
6. last it should provide a function to return the number forecasted values, it will take an input, forecasting horizion

In [3]:
!pip install torch 

  Using cached torch-1.13.1-cp39-none-macosx_10_9_x86_64.whl (135.3 MB)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class TimeSeriesForecastModel(nn.Module):
    
    def __init__(self, input_dim, hidden_dim=0, num_past_values=1, num_differences=0):
        super(TimeSeriesForecastModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_past_values = num_past_values
        self.num_differences = num_differences
        
        # initialize fully connected neural network layers for auto-regression
        if hidden_dim > 0:
            self.fc1 = nn.Linear(input_dim*num_past_values, hidden_dim)
            self.fc2 = nn.Linear(hidden_dim, input_dim)
        else:
            self.fc1 = nn.Linear(input_dim*num_past_values, input_dim)
        
        # initialize difference layers
        self.diff_layers = []
        for i in range(num_differences):
            self.diff_layers.append(nn.Linear(input_dim, input_dim))
        
    def forward(self, x):
        # apply differencing
        for layer in self.diff_layers:
            x = layer(x)
        
        # flatten input and apply auto-regression
        x = x.view(-1, self.input_dim*self.num_past_values)
        x = self.fc1(x)
        if self.hidden_dim > 0:
            x = nn.functional.relu(x)
            x = self.fc2(x)
        return x
    
    def fit(self, data, optimizer='adam', loss_function='mse', num_epochs=100):
        # prepare data by splitting into input and target variables
        x = data[:-1*self.num_past_values,:]
        y = data[self.num_past_values:,:]
        
        # initialize optimizer and loss function
        if optimizer == 'adam':
            optimizer = optim.Adam(self.parameters())
        elif optimizer == 'sgd':
            optimizer = optim.SGD(self.parameters(), lr=0.1)
        
        if loss_function == 'mse':
            loss_function = nn.MSELoss()
        elif loss_function == 'mae':
            loss_function = nn.L1Loss()
        
        # train model
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            y_pred = self(x)
            loss = loss_function(y_pred, y)
            loss.backward()
            optimizer.step()
            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Loss {loss.item()}')
        
    def predict(self, x, forecast_horizon):
        # apply differencing
        for layer in self.diff_layers:
            x = layer(x)
        
        # initialize prediction output
        y_pred = torch.zeros(forecast_horizon, self.input_dim)
        
        # forecast future values
        for i in range(forecast_horizon):
            x = x[-1*self.num_past_values:,:]
            y_next = self(x)
            y_pred[i,:] = y_next.detach().numpy()
            x = torch.cat((x[1:,:], y_next.unsqueeze(0)), axis=0)
            
        return y_pred


In [5]:
df=pd.read_csv("LTSF_Linear/LTSF-Linear/dataset/weather.csv").set_index("date")
df

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),...,wv (m/s),max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m�),PAR (�mol/m�/s),max. PAR (�mol/m�/s),Tlog (degC),OT
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:10:00,1008.89,0.71,273.18,-1.33,86.1,6.43,5.54,0.89,3.42,5.49,...,1.02,1.60,224.3,0.0,0.0,0.0,0.0,0.0,11.45,428.1
2020-01-01 00:20:00,1008.76,0.75,273.22,-1.44,85.2,6.45,5.49,0.95,3.39,5.45,...,0.43,0.84,206.8,0.0,0.0,0.0,0.0,0.0,11.51,428.0
2020-01-01 00:30:00,1008.66,0.73,273.21,-1.48,85.1,6.44,5.48,0.96,3.39,5.43,...,0.61,1.48,197.1,0.0,0.0,0.0,0.0,0.0,11.60,427.6
2020-01-01 00:40:00,1008.64,0.37,272.86,-1.64,86.3,6.27,5.41,0.86,3.35,5.37,...,1.11,1.48,206.4,0.0,0.0,0.0,0.0,0.0,11.70,430.0
2020-01-01 00:50:00,1008.61,0.33,272.82,-1.50,87.4,6.26,5.47,0.79,3.38,5.42,...,0.49,1.40,209.6,0.0,0.0,0.0,0.0,0.0,11.81,432.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 23:20:00,978.32,2.28,277.16,-0.80,80.0,7.20,5.76,1.44,3.67,5.89,...,0.73,1.40,180.6,0.0,0.0,0.0,0.0,0.0,13.40,433.0
2020-12-31 23:30:00,978.30,2.13,277.01,-0.43,83.1,7.12,5.92,1.20,3.77,6.05,...,0.43,0.82,174.0,0.0,0.0,0.0,0.0,0.0,13.42,439.6
2020-12-31 23:40:00,978.26,1.99,276.88,-0.71,82.2,7.05,5.80,1.26,3.69,5.93,...,0.38,0.76,248.9,0.0,0.0,0.0,0.0,0.0,13.45,435.2
